In [1]:
# Code to analyze emergency closure data

"""
Here are codes used by state inspectors when they determine
a restaurant should be closed temporarily. This is taken from
http://www.myfloridalicense.com/DBPR/hotels-restaurants/inspections/inspection-dispositions/


Facility Temporarily Closed:
Operations ordered stopped until violations are corrected
The inspector recommended closing the facility immediately
after finding conditions that may endanger the health and
safety of the public.

Dispositions included in this result are:

Emergency order recommended – Conditions have been found
that endanger the health and safety of the public requiring
immediate closure of the establishment.

Administrative determination recommended – The establishment
is operating without a license and action is being taken
to ensure proper licensing is completed.

Emergency Order Callback Not Complied – Corrections to violations
that resulted in an emergency order were not completed
at the time of inspection. Violations may not be noted
again on these inspection reports.
"""

import csv
import pandas as pd
import numpy as np
import sqlite3

# Set up connection to database.

imacpath = "/Users/rayd/workspace/flinsp/datafiles/"
airpath = "/Users/Doug/workspace/flinsp/datafiles/"
dbfile = "rinspect18.sqlite"
dbpath = airpath + dbfile
conn = sqlite3.connect(dbpath)

# Make a list of visitid numbers for restaurants shut down
# by inspectors.

conn.row_factory = lambda cursor, row: row[0]
c = conn.cursor()

# on initial inspection
vids = c.execute("SELECT visitid FROM fdinsp WHERE inspdispos = 'Emergency order recommended'").fetchall()

# remained shut on subsequent inspection
vids2 = c.execute("SELECT visitid FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Not Complied'").fetchall()

# Make pandas dataframe for both sets

conn = sqlite3.connect(dbpath)

df = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos = 'Emergency order recommended';", conn)
df2 = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Not Complied';", conn)

# Test to see if our dataset is complete

count1 = df.shape
count2 = df2.shape
print("There were " + str(count1[0]) + " restaurants closed on first inspection.")
print("\nThere were " + str(count2[0]) + " remained closed on subsequent inspection.")

# Contains 'Emergency' but means reopened
df_test1 = pd.read_sql_query(
    "SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Complied';", conn
    )
count3 = df_test1.shape
print("\nThere were " + str(count3[0]) + " cleared and opened by subsequent inspection.")

# Contains 'Emergency' but also means reopened
df_test2 = pd.read_sql_query(
    "SELECT * FROM fdinsp WHERE inspdispos = 'Emergency Order Callback Time Extension';", conn
    )
count4 = df_test2.shape
print("\nThere were " + str(count4[0]) + " reopened but will need another inspection.")

print("\n" +
    str(count1[0]) + " + " +
    str(count2[0]) + " + " +
    str(count3[0]) + " + " +
    str(count4[0]) + " + " +
    " = " + str(count1[0] + count2[0] + count3[0] + count4[0])
     )

# Contains something like 'Emergency' but are there some where spelling or capitalization shifts?
df_test3 = pd.read_sql_query("SELECT * FROM fdinsp WHERE inspdispos LIKE '%mergency%';", conn)
count5 = df_test3.shape
print("\nThere were " + str(count5[0]) + " that had some word like 'emergency'.")

print("\nSo it looks like we got them all.")


There were 1238 restaurants closed on first inspection.

There were 547 remained closed on subsequent inspection.

There were 810 cleared and opened by subsequent inspection.

There were 365 reopened but will need another inspection.

1238 + 547 + 810 + 365 +  = 2960

There were 2960 that had some word like 'emergency'.

So it looks like we got them all.


In [2]:
# Create list of dictionaries with detailed inspection reports
# that led to closures

def dict_factory(cursor, row):
    dvio = {}
    for idx, col in enumerate(cursor.description):
        dvio[col[0]] = row[idx]
    return dvio

lvio = []
lvio2 = []

for vid in vids:
    con = sqlite3.connect(dbpath)
    con.row_factory = dict_factory
    cur = con.cursor()
    cur.execute(f"SELECT * FROM violations WHERE visitid = {vid}")
    lvio.extend(cur.fetchall())
    con.close()

for vid2 in vids2:
    con = sqlite3.connect(dbpath)
    con.row_factory = dict_factory
    cur = con.cursor()
    cur.execute(f"SELECT * FROM violations WHERE visitid = {vid2}")
    lvio2.extend(cur.fetchall())
    con.close()


In [3]:
# Make dataframes with violation details

df3 = pd.DataFrame(lvio) # Closed on initial inspection
df4 = pd.DataFrame(lvio2) # Remained closed after follow-up

# What was the most common violation in a closure inspection?

df3g = df3.groupby('violation').count().sort_values(by=['visitid'], axis=0, ascending=False)

# What were the most common violations in a closure inspection?

df3.groupby('violation').count().sort_values(
    by=['visitid'], axis=0, ascending=False
    ).head(10)


,details_id,id,obs,visitid
violation,,,,
35A-05-4,541,541,541,541
35A-03-4,466,466,466,466
03A-02-4,410,410,410,410
35A-04-4,383,383,383,383
35A-02-5,277,277,277,277
31B-02-4,239,239,239,239
01B-02-4,203,203,203,203
31A-03-4,197,197,197,197
02C-02-4,179,179,179,179


In [19]:
df_stopsale = df3[df3.obs.str.contains("Stop Sale issued")==True]
df_stopsale.shape
print("In the df of inspections using the summary reports that contained")
print("the words 'Emergency order recommended', we saw " + str(count1[0]) + " initial closures." )
print("But in checking the detailed reports, we find " + str(df_stopsale.shape[0]) + " with 'Stop sale'")
print("Can you have a closure without a 'Stop sale'?")
print("Answers: Yes. 'Stop sale' refers to food item, maybe bad temp, not to the restaurant generally.")

In the df of inspections using the summary reports that contained
the words 'Emergency order recommended', we saw 1238 initial closures.
But in checking the detailed reports, we find 303 with 'Stop sale'
Can you have a closure without a 'Stop sale'?
Answers: Yes. 'Stop sale' refers to food item, maybe bad temp, not to the restaurant generally.


In [17]:
df.head()

,librow,county,licnum,sitename,streetaddy,cityaddy,zip,inspnum,insptype,inspdispos,inspdate,totalvio,highvio,licid,visitid,time_now,time_posted
0,85,Broward,1601438,Tarks Of Dania,1317 S Federal Hwy,Dania,33004-4361,2826990,Complaint Full,Emergency order recommended,"2018, 07, 09",27,6,2137236,6673447,None,2018-10-07 08:37
1,204,Broward,1609502,Saveur Tropical Restaurant,515 NE 24 St,Pompano Beach,33064,2822052,Routine - Food,Emergency order recommended,"2018, 09, 24",13,5,7033019,6646049,None,2018-10-07 08:37
2,244,Broward,1610123,Betty's Rest & BBQ,601 NW 22 Road,Fort Lauderdale,33311-7756,2831162,Complaint Full,Emergency order recommended,"2018, 08, 08",9,1,2142981,6690254,None,2018-10-07 08:37
3,262,Broward,1610222,Jade Garden Restaurant,244 S University Dr,Plantation,33324,2834551,Complaint Full,Emergency order recommended,"2018, 08, 15",29,7,2447780,6703478,None,2018-10-07 08:37
4,623,Broward,1616508,Fortune Dragon,7178 N University Dr,Tamarac,33321,2789426,Routine - Food,Emergency order recommended,"2018, 08, 28",16,3,3379484,6595847,None,2018-10-07 08:37


In [4]:
# Can we find which violations resulted in the most closures?

# Violation codes and descriptions
a = '35A-01-4' # Intermediate: Service animals
b = '35A-02-5' # High priority: Live, small flying insects in food service area
c = '35A-03-4' # Basic: Dead roaches on premesis
d = '35A-04-4' # High priority: Rodent activity present as evidenced by droppings
e = '35A-05-4' # High priority: Live roaches found
f = '35A-06-4' # Basic: Accumulation of dead or trapped pests
g = '35A-07-4' # High priority: Small flying insects in bar, kitchen, dumster, prep area
h = '35A-09-4' # High priority: Presence of insects, rodents or other pests
i = '35A-18-4' # High priority: Rodent rub marks present
j = '35A-20-4' # Basic: Dead rodent present
k = '35A-21-4' # High priority: Rodent burrow or nesting materials present
l = '35A-23-4' # High priority: Rodent droppings present
m = '03A-02-4' # High priority: Potentially hazardous  cold food held at greater than 41 degrees


In [35]:
# Are there any closures that don't involve 35A-0*'?
df3z = df3
df3za = df3.groupby('visitid').filter(lambda x: all(a != i for i in x['violation']))
df3zb = df3z.groupby('visitid').filter(lambda x: all(b != i for i in x['violation']))
df3zc = df3z.groupby('visitid').filter(lambda x: all(c != i for i in x['violation']))
df3zd = df3z.groupby('visitid').filter(lambda x: all(d != i for i in x['violation']))
df3ze = df3z.groupby('visitid').filter(lambda x: all(e != i for i in x['violation']))
df3zf = df3z.groupby('visitid').filter(lambda x: all(f != i for i in x['violation']))
df3zg = df3z.groupby('visitid').filter(lambda x: all(g != i for i in x['violation']))
df3zh = df3z.groupby('visitid').filter(lambda x: all(h != i for i in x['violation']))
df3zi = df3z.groupby('visitid').filter(lambda x: all(i != i for i in x['violation']))
df3zj = df3z.groupby('visitid').filter(lambda x: all(j != i for i in x['violation']))
df3zk = df3z.groupby('visitid').filter(lambda x: all(k != i for i in x['violation']))
df3zl = df3z.groupby('visitid').filter(lambda x: all(l != i for i in x['violation']))
df3zm = df3z.groupby('visitid').filter(lambda x: all(m != i for i in x['violation']))

#On checking database directly, this doesn't seem to work ....
print("This approach finds whether, if a violation is present, it always")
print("results in a closure order:")
print("\nThe number of closures on initial inspection = " + str(df3z.shape[0]))
print("\nThe number of closures, despite this being found:")
print("'Service animals' = " + str(df3za.shape[0]))
print("'Live, small flying insects in food service area' = " + str(df3zb.shape[0]))
print("'Dead roaches on premesis' = " + str(df3zc.shape[0]))
print("'Rodent activity present as evidenced by droppings' = " + str(df3zd.shape[0]))
print("'Live roaches found' = " + str(df3ze.shape[0]))
print("'Accumulation of dead or trapped pests' = " + str(df3zf.shape[0]))
print("'Small flying insects in bar, kitchen, dumster, prep area' = " + str(df3zg.shape[0]))
print("'Presence of insects, rodents or other pests' = " + str(df3zh.shape[0]))
print("'Rodent rub marks present' = " + str(df3zi.shape[0]))
print("'Dead rodent present' = " + str(df3zj.shape[0]))
print("'Rodent burrow or nesting materials present' = " + str(df3zk.shape[0]))
print("'Rodent droppings present' = " + str(df3zl.shape[0]))
print("'Potentially hazardous  cold food held at greater than 41 degrees' = " + str(df3zm.shape[0]))
print("\nSo, if 'Rodent rub marks present', the doors are closed!")


This approach finds whether, if a violation is present, it always
results in a closure order:

The number of closures on initial inspection = 16625

The number of closures, despite this being found:
'Service animals' = 16604
'Live, small flying insects in food service area' = 11548
'Dead roaches on premesis' = 8358
'Rodent activity present as evidenced by droppings' = 10255
'Live roaches found' = 7557
'Accumulation of dead or trapped pests' = 16625
'Small flying insects in bar, kitchen, dumster, prep area' = 16006
'Presence of insects, rodents or other pests' = 15831
'Rodent rub marks present' = 0
'Dead rodent present' = 16250
'Rodent burrow or nesting materials present' = 16118
'Rodent droppings present' = 15224
'Potentially hazardous  cold food held at greater than 41 degrees' = 8477

So, if 'Rodent rub marks present', the doors are closed!


In [22]:
# Which counties had the most closures?
# Calculated as closers per licensed restaurant

# Count closures per county
dfc = df.groupby('county').count()
dfc = dfc.licnum.reset_index()
dfc = dfc.rename(columns={'county' : 'county', 'licnum' : 'closures'})
dfc = dfc.set_index('county')
print(dfc)

# Which counties are included in closures
co_inc = list(df.groupby(['county']).groups.keys())

#List of all Florida counties
with open('outfiles/counties.txt', 'r') as f:
    fl_counties = [line.rstrip('\n') for line in f]

def diff(co_inc, fl_counties):
    co_dif = [i for i in co_inc + fl_counties if i not in co_inc]
    return co_dif

missing_counties = diff(co_inc, fl_counties)

print("\nDid any counties not have closure orders in FY2018-19?")
print("\nThese are not included: " + str(', '.join(missing_counties)))
print("\nBut Miami-Dade listed simply as Dade in our data frame.")

missing = list(missing_counties)
missing.remove('Miami-Dade')


              closures
county                
Alachua             14
Baker                2
Bay                 19
Bradford             2
Brevard             48
Broward            114
Charlotte            5
Citrus              10
Clay                 4
Collier             17
Columbia             7
Dade               145
Dixie                1
Duval              108
Escambia            19
Flagler              2
Franklin             2
Gadsden              6
Gilchrist            2
Glades               1
Hardee               1
Hendry               1
Hernando            23
Highlands            7
Hillsborough        88
Indian River         4
Jackson              5
Jefferson            1
Lake                16
Lee                 28
Leon                32
Levy                 1
Madison              1
Manatee             27
Marion              44
Martin               1
Monroe               7
Okaloosa             9
Okeechobee           3
Orange              75
Osceola              4
Palm Beach 

In [28]:
# Read in csv of licensed restaurants per county
df_cntylic = pd.read_csv('countycount.csv')
df_cntylic = df_cntylic.drop(['Unnamed: 0'], axis=1)
df_cntylic = df_cntylic[~df_cntylic['co_name'].isin(missing)] # drop missing counties
df_cntylic= df_cntylic.rename(columns={"lic_count": "licenses", "co_name": "county"})
df_cntylic = df_cntylic.set_index('county')


In [29]:
# Closures per license
dfc = df_cntylic.join(dfc)


In [30]:
dfc['ratio'] = dfc.closures / dfc.licenses
dfc['percent'] = dfc.ratio * 100
dfc = dfc.sort_values(by=['ratio'])
most_closed = dfc.sort_values(by=['ratio'], ascending=False).head(10)
least_closed  = dfc.sort_values(by=['ratio'], ascending=True).head(10)

In [33]:
most_closed.head(20)
print("Counties as outliers for the most closures:")
print(most_closed.head(20))

Counties as outliers for the most closures:
           licenses  closures     ratio    percent
county                                            
Gilchrist        16         2  0.125000  12.500000
Glades           15         1  0.066667   6.666667
Gadsden          91         6  0.065934   6.593407
Hernando        366        23  0.062842   6.284153
Marion          734        44  0.059946   5.994550
Dixie            19         1  0.052632   5.263158
Jackson          95         5  0.052632   5.263158
Wakulla          58         3  0.051724   5.172414
Baker            44         2  0.045455   4.545455
Columbia        162         7  0.043210   4.320988


In [34]:
most_closed.head(20)
print("Counties as outliers for the least closures:")
print(least_closed.head(20))

Counties as outliers for the least closures:
              licenses  closures     ratio   percent
county                                              
Martin             526         1  0.001901  0.190114
Osceola           1050         4  0.003810  0.380952
Walton             335         2  0.005970  0.597015
Flagler            233         2  0.008584  0.858369
Seminole          1042        10  0.009597  0.959693
Indian River       391         4  0.010230  1.023018
Monroe             677         7  0.010340  1.033973
Clay               370         4  0.010811  1.081081
Levy                89         1  0.011236  1.123596
Hendry              82         1  0.012195  1.219512


In [37]:
df.head()

,librow,county,licnum,sitename,streetaddy,cityaddy,zip,inspnum,insptype,inspdispos,inspdate,totalvio,highvio,licid,visitid,time_now,time_posted
0,85,Broward,1601438,Tarks Of Dania,1317 S Federal Hwy,Dania,33004-4361,2826990,Complaint Full,Emergency order recommended,"2018, 07, 09",27,6,2137236,6673447,None,2018-10-07 08:37
1,204,Broward,1609502,Saveur Tropical Restaurant,515 NE 24 St,Pompano Beach,33064,2822052,Routine - Food,Emergency order recommended,"2018, 09, 24",13,5,7033019,6646049,None,2018-10-07 08:37
2,244,Broward,1610123,Betty's Rest & BBQ,601 NW 22 Road,Fort Lauderdale,33311-7756,2831162,Complaint Full,Emergency order recommended,"2018, 08, 08",9,1,2142981,6690254,None,2018-10-07 08:37
3,262,Broward,1610222,Jade Garden Restaurant,244 S University Dr,Plantation,33324,2834551,Complaint Full,Emergency order recommended,"2018, 08, 15",29,7,2447780,6703478,None,2018-10-07 08:37
4,623,Broward,1616508,Fortune Dragon,7178 N University Dr,Tamarac,33321,2789426,Routine - Food,Emergency order recommended,"2018, 08, 28",16,3,3379484,6595847,None,2018-10-07 08:37


In [125]:
# List restaurants by how many closures they had this year.

# Drop unneded columns
dropcols = ['librow', 'inspnum', 'insptype', 'inspdispos', 'inspdate', 'totalvio', 'highvio', 'licid', 'visitid', 'time_now', 'time_posted']
df_restaurants = df.drop(dropcols, axis=1)
df_restaurants.head()

,county,licnum,sitename,streetaddy,cityaddy,zip
0,Broward,1601438,Tarks Of Dania,1317 S Federal Hwy,Dania,33004-4361
1,Broward,1609502,Saveur Tropical Restaurant,515 NE 24 St,Pompano Beach,33064
2,Broward,1610123,Betty's Rest & BBQ,601 NW 22 Road,Fort Lauderdale,33311-7756
3,Broward,1610222,Jade Garden Restaurant,244 S University Dr,Plantation,33324
4,Broward,1616508,Fortune Dragon,7178 N University Dr,Tamarac,33321


In [140]:
#Count repeated closures per restaurant, listed by license number
dupes = df_restaurants.pivot_table(index=['licnum'],aggfunc='size')
dupes = dupes.reset_index()

# Drop repeated rows so we can match with the dupes
df_repeats = df_restaurants[df_restaurants.duplicated()]

# Merge dataframes to get df with count of repeats
dupes_count = pd.merge(df_repeats, dupes, on='licnum')
dupes_count = dupes_count.sort_values(by=[0], ascending=False)
dupes_count = dupes_count[dupes_count.duplicated()]
dupes_count

,county,licnum,sitename,streetaddy,cityaddy,zip,0
33,Marion,5202788,Pineapple Sam's Cafe,18976 E Hwy 40,Silver Springs,34488,4
67,Marion,5200016,Richard's Place,316 E Silver Spgs Blvd,Ocala,344705830,4
34,Marion,5202788,Pineapple Sam's Cafe,18976 E Hwy 40,Silver Springs,34488,4
65,Marion,5200016,Richard's Place,316 E Silver Spgs Blvd,Ocala,344705830,4
79,Broward,1620527,Asian Buffet & Grill,240 S Federal Highway,Deerfield Beach,33441,3
52,Orange,5805454,New York Broadway Pizza,"6807 Visitors Cir , Suite B",Orlando,32819,3
94,Pinellas,6217825,Hideaway Pizza Pub,3901 6 St S,St. Petersburg,33705,3
97,Polk,6308173,Marco's Pizza,703 N Combee Road,Lakeland,33801,3
23,Leon,4703122,New Times Country Buffet,1701 N Monroe St,Tallahassee,32303,3
99,Pinellas,6212759,Los Mariaches #2,9100 Ulmerton Road,Largo,337713821,3


,licnum,0
717,5202788,4
691,5200016,4
756,5805454,3
647,4703122,3
984,6217413,3
398,2613227,3
933,6212759,3
908,6150545,3
1014,6308173,3
637,4702440,3


In [40]:
# NO NEED TO RUN EACH TIME

# Write restaurants with initial closures to csv 
df_restaurants = df_restaurants.drop_duplicates()
df_restaurants.to_csv('outfiles/closedrestaurants.csv', index=False)

# Write csv files for violation details

keys = lvio[0].keys
with open('outfiles/closurevios.csv', 'w', newline='') as output_file:
    fc = csv.DictWriter(output_file,
                        fieldnames=lvio[0].keys()
                       )
    fc.writeheader()
    fc.writerows(lvio)

keys = lvio2[0].keys
with open('outfiles/closurevios2.csv', 'w', newline='') as output_file:
    fc = csv.DictWriter(output_file,
                        fieldnames=lvio2[0].keys()
                       )
    fc.writeheader()
    fc.writerows(lvio2)


In [ ]:
# Make a list of all violations involved in initial closures
clist = df3.violation.values.tolist()
clist = list(dict.fromkeys(clist))
clist